In [1]:
# organize imports
import cv2
# import imutils
import numpy as np
from sklearn.metrics import pairwise
from keras.models import load_model
# from scipy.misc import imresize

# global variables
bg = None

In [2]:
def run_avg(image, accumWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, accumWeight)

In [3]:
# def segment(image, accumWeight=0.5, threshold=25):
#     global bg
#     # Background Subtraction with Dynamic Background Updating
#     if bg is None:
#         bg = image.copy().astype("float")
#         return None
    
#     # compute weighted average, accumulate it, and update the background
#     cv2.accumulateWeighted(image, bg, accumWeight)

#     # find the absolute difference between background and current frame
#     diff = cv2.absdiff(bg.astype("uint8"), image)

#     # Adaptive Thresholding
#     thresholded = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#     # Noise Reduction using Morphological Operations
#     kernel = np.ones((5, 5), np.uint8)
#     thresholded = cv2.morphologyEx(thresholded, cv2.MORPH_OPEN, kernel)

#     # get the contours in the thresholded image
#     contours, _ = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     # return None if no contours detected
#     if len(contours) == 0:
#         return None
#     else:
#         # based on contour area, get the maximum contour which is the hand
#         segmented = max(contours, key=cv2.contourArea)
#         return (thresholded, segmented)

In [4]:
import cv2
import numpy as np

def segment(image, bg, accumWeight, threshold=25):
    # Find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # Apply adaptive thresholding to dynamically determine the threshold
    thresholded = cv2.adaptiveThreshold(diff, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Perform morphological opening to reduce noise and smoothen the image
    kernel = np.ones((5,5),np.uint8)
    opening = cv2.morphologyEx(thresholded, cv2.MORPH_OPEN, kernel)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(opening.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on area to remove small noise regions
    min_contour_area = 1000
    significant_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]

    # Return None if no significant contours are detected
    if len(significant_contours) == 0:
        return
    else:
        # Get the contour with the largest area (assumed to be the hand)
        segmented = max(significant_contours, key=cv2.contourArea)
        return (opening, segmented)

# Usage example:
# bg = None  # Initialize background
# accumWeight = 0.5  # Weight for accumulating background
# image = cv2.imread('your_image.jpg')  # Read input image
# thresholded, segmented = segment(image, bg, accumWeight)
# cv2.imshow('Thresholded', thresholded)
# cv2.waitKey(0)


In [5]:
# load Model Weights

def _load_weights():
    try:
        model = load_model("hand_gesture_recognition.h5")
        print(model.summary())
        # print(model.get_weights())
        # print(model.optimizer)
        return model
    except Exception as e:
        return None


    
def getPredictedClass(model):
    if model is None:
        print("Model is not loaded successfully.")
        return None

    image = cv2.imread('Temp.png')
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image = cv2.resize(gray_image, (100, 120))

    gray_image = gray_image.reshape(1, 100, 120, 1)

    prediction = model.predict_on_batch(gray_image)

    predicted_class = np.argmax(prediction)
    if predicted_class == 0:
        return "Blank"
    elif predicted_class == 1:
        return "OK"
    elif predicted_class == 2:
        return "Thumbs Up"
    elif predicted_class == 3:
        return "Thumbs Down"
    elif predicted_class == 4:
        return "Punch"
    elif predicted_class == 5:
        return "High Five"


if __name__ == "__main__":
    # initialize accumulated weight
    accumWeight = 0.5

    # get the reference to the webcam
    camera = cv2.VideoCapture(0)

    fps = int(camera.get(cv2.CAP_PROP_FPS))
    # region of interest (ROI) coordinates
    top, right, bottom, left = 10, 350, 225, 590
    # initialize num of frames
    num_frames = 0
    # calibration indicator
    calibrated = False
    model = _load_weights()
    k = 0
    # keep looping, until interrupted
    while (True):
        # get the current frame
        (grabbed, frame) = camera.read()

        # resize the frame
        frame = cv2.resize(frame, (700,700))
        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the height and width of the frame
        (height, width) = frame.shape[:2]

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our weighted average model gets calibrated
        if num_frames < 30:
            run_avg(gray, accumWeight)
            if num_frames == 1:
                print("[STATUS] please wait! calibrating...")
            elif num_frames == 29:
                print("[STATUS] calibration successfull...")
        else:
            # segment the hand region
            hand = segment(gray,bg,accumWeight)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand

                # draw the segmented region and display the frame
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))

                # count the number of fingers
                # fingers = count(thresholded, segmented)
                if k % (fps / 6) == 0:
                    cv2.imwrite('Temp.png', thresholded)
                    predictedClass = getPredictedClass(model)
                    cv2.putText(clone, str(predictedClass), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                # show the thresholded image
                cv2.imshow("Thesholded", thresholded)
                
        k = k + 1
        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0, 255, 0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break

    # free up memory
    camera.release()
    cv2.destroyAllWindows()




Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 98, 118, 32)       320       
                                                                 
 batch_normalization_3 (Bat  (None, 98, 118, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 49, 59, 32)        0         
 g2D)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 49, 59, 32)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 47, 57, 64)        18496     
                                                                 
 batch_normalization_4 (Bat  (None, 47, 57, 64)     